<a href="https://colab.research.google.com/github/ekaterinaKrupkina/decomposer_notebooks/blob/main/separate_steps/experiments_generate_completions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
import subprocess
import json
from google.colab import userdata
import shlex
import pandas as pd
import time
import sys
from IPython.display import clear_output


TODO:


1.   func to generate test cases
3.   think how to pass paths to the funcs





## Set up environment

## After installation you'll be asked to reload the runtime. Please, REFUSE


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
REQUIREMENTS = """
confection
fschat
ray
openai==1.34.0
anthropic==0.28.0
mistralai
google-generativeai
google-cloud-aiplatform
matplotlib
weave==0.50.5
pandas
baselines
accelerate
fastapi==0.111.0
pydantic==2.7.3
tiktoken==0.7.0
aiohttp==3.9.5
aiofiles==23.2.1
mistralai==0.1.8
huggingface-hub
blobfile
transformers
"""

In [ ]:
with open("requirements.txt", "w") as file:
    file.write(REQUIREMENTS)

In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.7/862.7 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.3/123.3 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.6

In [ ]:
os.remove("requirements.txt")

In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


### Set variables

* WANDB_API_KEY
* GITHUB_TOKEN
* OPENAI_API_KEY
* MISTRAL_API_KEY
* CLAUDE_API_KEY
* HF_TOKEN

In [ ]:
# wandb & badlamma setting
os.environ["WANDB__EXECUTABLE"] = "/usr/local/bin/python"
os.environ["BADLLAMA_ENDPOINT_URL"] = "https://tl056txanw7ohnrj.us-east-1.aws.endpoints.huggingface.cloud/v1/"

In [ ]:
def set_env_variable(variable_name):
    """
    Sets an environment variable with the specified name. If the variable is already present
    in the environment, it uses the existing value from `userdata`. If not, it prompts the user
    to input the value and sets the environment variable with that input.

    Args:
        variable_name (str): The name of the environment variable to be set.

    Raises:
        KeyError: If the variable_name is not present in `userdata` when it is expected.
    """
    if userdata.get(variable_name):
       os.environ[variable_name] = userdata.get(variable_name)
    else:
       os.environ[variable_name] = input(f"Enter the value for {variable_name}: ")

In [ ]:
set_env_variable("OPENAI_API_KEY")
set_env_variable("WANDB_API_KEY")
set_env_variable("GITHUB_TOKEN")

## Clone repositories

In [ ]:
if not os.path.exists("HarmBench"):
  !git clone -q --recursive https://github.com/ekaterinaKrupkina/HarmBench.git
%cd /content/drive/MyDrive/HarmBench

/content/drive/MyDrive/HarmBench


In [ ]:
if not os.path.exists("Decomposition-Harness"):
  !git clone https://{userdata.get("GITHUB_TOKEN")}@github.com/PalisadeResearch/Decomposition-Harness

In [ ]:
if not os.path.exists("decomposer_notebooks"):
  !git clone -q --recursive https://github.com/ekaterinaKrupkina/decomposer_notebooks.git

In [ ]:
!sed -i 's#API_TIMEOUT = 20#API_TIMEOUT = 800#' /content/drive/MyDrive/HarmBench/baselines/pair/language_models.py

In [ ]:
!cp -f /content/drive/MyDrive/HarmBench/decomposer_notebooks/separate_steps/generate_completions.py /content/drive/MyDrive/HarmBench/generate_completions.py

In [ ]:
!cp -f /content/drive/MyDrive/HarmBench/decomposer_notebooks/separate_steps/model_utils.py /content/drive/MyDrive/HarmBench/baselines/model_utils.py

## Generate Test Cases

In [ ]:
def generate_test_cases(behaviors_path="/content/drive/MyDrive/HarmBench/data/behavior_datasets/harmbench_behaviors_text_all.csv",
                        test_cases_save_dir="/content/drive/MyDrive/HarmBench/results/test_cases",
                        experiment_name="default"):
  """
    Generates test cases by running an external script with specified parameters. This function
    constructs a command to call a Python script named `generate_test_cases.py` and executes it
    using the provided parameters for behaviors path, test cases save directory, and experiment name.

    Args:
        behaviors_path (str, optional): The file path to the behaviors dataset. Defaults to "/content/drive/MyDrive/HarmBench/data/behavior_datasets/harmbench_behaviors_text_all.csv".
        test_cases_save_dir (str, optional): The directory where the generated test cases will be saved. Defaults to "/content/drive/MyDrive/HarmBench/results/test_cases".
        experiment_name (str, optional): The name of the experiment for which test cases are being generated. Defaults to "default".

    Returns:
        None
  """

  command = [
        "python", "-u", "generate_test_cases.py",
        "--method_name=DirectRequest",
        f"--experiment_name={experiment_name}",
        f"--behaviors_path={behaviors_path}",
        f"--save_dir={test_cases_save_dir}"
    ]

  process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

  for line in process.stdout:
      print(line, end='')

  return_code = process.wait()

  if return_code != 0:
      print(f"Process exited with return code {return_code}")



## Make models config

In [ ]:
models_config = """
claude-3-opus-20240229:
  model:
    model_name_or_path: gpt-4o
    token: <your_anthropic_token>
  model_type: closed_source

gpt-4o-2024-05-13:
  model:
    model_name_or_path: gpt-4o
    token: <your_openai_token>
  model_type: closed_source

gpt-3.5-turbo-1106:
  model:
    model_name_or_path: gpt-3.5-turbo
    token: <your_openai_token>
  model_type: closed_source

badllama_70B:
  model:
    model_name_or_path: gpt-4o
    token: <your_openai_token>
  num_gpus: 1
  model_type: open_source
"""



In [ ]:
import yaml

models_config_dict = yaml.safe_load(models_config)

with open('models_config.yaml', 'w') as file:
    yaml.dump(models_config_dict, file, default_flow_style=False)

print("models_config.yaml file has been created.")

models_config.yaml file has been created.


## Some important funcs

In [ ]:
def run_in_background(script_path, output_file='output.log'):
    """
    Run a Python script in the background and redirect its output to a file.

    This function executes the specified Python script as a separate process,
    allowing it to run independently of the calling process. The script's
    standard output and standard error are redirected to the specified output file.

    Parameters:
    script_path (str): The path to the Python script to be executed.
    output_file (str, optional): The path to the file where the script's output
                                 will be written. Defaults to 'output.log'.

    Returns:
    int: The process ID (PID) of the newly created background process.

    Notes:
    - The function uses subprocess.Popen to create a new process.
    - shlex.split is used to properly parse the command arguments.
    - The script's stderr is redirected to stdout, which is then written to the output file.
    - The process is set to run independently of the parent process using os.setsid.
    - The function prints the PID of the new process to the console.

    Raises:
    subprocess.SubprocessError: If there's an error in creating or running the subprocess.
    IOError: If there's an error in opening or writing to the output file.

    Example:
    >>> pid = run_in_background('/path/to/script.py', 'script_output.log')
    >>> print(f"Script is running in background with PID: {pid}")
    """
    # Construct the command
    command = f"python {script_path}"

    with open(output_file, 'a') as f:
        process = subprocess.Popen(
            shlex.split(command),  # Use shlex.split for proper argument parsing
            stdout=f,
            stderr=subprocess.STDOUT,  # Redirect stderr to stdout
            preexec_fn=os.setsid  # This allows the process to run independently of the parent
        )

    print(f"Process started with PID: {process.pid}")
    return process.pid

In [ ]:
def run_generate_completions(model_name, test_cases_path, save_path, behaviors_path, models_config_file="models_config.yaml"):
    """
    Run the generate_completions.py script as a subprocess.

    This function executes the generate_completions.py script with the specified parameters,
    captures its output in real-time, and prints it to the console.

    Args:
        model_name (str): Name of the model to use for generating completions.
        test_cases_path (str): Path to the test cases file.
        save_path (str): Path where the generated completions will be saved.
        behaviors_path (str): Path to the behaviors dataset file.
        models_config_file (str): Path to the models configuration file. By default, it is set to "models_config.yaml".

    Returns:
        None

    Raises:
        subprocess.CalledProcessError: If the subprocess returns a non-zero exit status.
    """
    command = [
        "python", "-u", "generate_completions.py",
        f"--model_name={model_name}",
        f"--models_config_file={models_config_file}",
        f"--test_cases_path={test_cases_path}",
        f"--save_path={save_path}",
        f"--behaviors_path={behaviors_path}"
    ]

    process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

    for line in process.stdout:
        print(line, end='')

    return_code = process.wait()

    if return_code != 0:
        print(f"Process exited with return code {return_code}")

In [ ]:
def experiment(exp):
    """
    Run a complete experiment pipeline including completion generation, evaluation, and result processing.

    This function sets up the environment, runs the necessary scripts for generating completions
    and evaluating them, and then processes and logs the results.

    Args:
        experiment_params (dict): A dictionary containing experiment parameters including model names.

    Returns:
        None
    """
    !pkill -9 --full 'web\.py'
    time.sleep(1)

    for k, v in exp['params'].items():
        os.environ[k] = str(v)

    os.environ['BIND_PORT'] = exp['BIND_PORT']
    MODEL_TARGET = os.getenv("MODEL_TARGET")
    MODEL_DECOMPOSER = os.getenv("MODEL_DECOMPOSER")

    os.environ["OPENAI_BASE_URL"] = "https://api.openai.com/v1"
    run_in_background("./Decomposition-Harness/server/web.py", "output.log")

    os.environ["OUTPUT_FILES_PREFIX"] = f'{MODEL_TARGET}_{MODEL_DECOMPOSER}'

    time.sleep(10) # waiting for decomposer server initialization

    os.environ["OPENAI_BASE_URL"] = exp['url']

    try:
      run_generate_completions(
          model_name=MODEL_TARGET,
          test_cases_path="/content/drive/MyDrive/HarmBench/results/test_cases/test_cases.json",
          save_path=f"./results/completions/completions_{MODEL_TARGET}_{MODEL_DECOMPOSER}_full.json",
          behaviors_path="./data/behavior_datasets/harmbench_behaviors_text_all.csv"
      )

    except Exception as e:
      print(e)
      pass

    for k, v in exp['params'].items():
      del os.environ[k]




In [ ]:
!pkill -9 --full 'web\.py'

## Run experiment

In [ ]:
experiments = [
    # {
    #     'url': 'http://127.0.0.1:8082',
    #     'BIND_PORT': '8082',
    #     'params': {
    #         "MODEL_TARGET": "gpt-4o-2024-05-13",
    #         "MODEL_COMPOSER": "badllama_70B",
    #         "MODEL_DECOMPOSER": "badllama_7B",
    #         "MODEL_SURROGATE": "gpt-3.5-turbo-1106"
    #     }
    # },
      {
        'url': 'http://127.0.0.1:8084',
        'BIND_PORT': '8084',
        'params': {
            "MODEL_TARGET": "claude-3-opus-20240229",
            "MODEL_DECOMPOSER": "",
            "MAX_QUERY_RETRIES_TARGET": "1"
        }
    },
    # {
    #     'url': 'http://127.0.0.1:8085',
    #     'BIND_PORT': '8085',
    #     'params': {
    #         "MODEL_TARGET": "gpt-3.5-turbo-1106",
    #         "MODEL_DECOMPOSER": "",
    #         "MAX_QUERY_RETRIES_TARGET": "1"
    #     }
    # }
]


In [ ]:
%%time

for exp in experiments:
  experiment(exp)
  print('======================================')
print("===================END=======================")

Process started with PID: 46647
2024-07-19 09:30:58.725219: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 09:30:58.725276: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 09:30:58.726238: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-19 09:30:58.731617: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-19 09:30:59.8

In [34]:
clear_output()

In [33]:
from google.colab import output
output.clear()



---

end of file